<div class='alert-block alert-info'>
    <br>
    <h1 align="center"> AI Project 
    <h3 align="center">Nadia Godje </h3>
    <h3 align="center">Stive </h3>
    <\br>
</div>

In [5]:
import pandas as pd
import numpy as np
defects=pd.read_csv("defects.csv", sep=",")
defects


,x,class
0,355.449335,c
1,92.496236,a
2,141.876795,c
3,431.833902,c
4,435.028461,c
...,...,...
495,171.161186,a
496,365.899255,a
497,157.372180,b
498,275.538229,b


On implémente d'abord un problème de CSP

On crée d'abord le problème csp que l'on veut résoudre

def create_biscuits_csp():
    biscuit0,biscuit1,biscuit2,biscuit3= 'B0', 'B1', 'B2', 'B3'
    lenghts = [4, 8, 2, 5]
    values = [6, 12, 1, 8]

    variables = [biscuit0,biscuit1,biscuit2,biscuit3]

    def constraint_function(first_variable, first_value, second_variable, second_value):
        return first_value != second_value

    # Define the set of constraints
    constraints = {
        biscuit0: constraint_function,
        biscuit1: constraint_function,
        biscuit2: constraint_function,
        biscuit3: constraint_function,
    }

    return CSP(variables, domains, neighbours, constraints)

if __name__ == '__main__':
    SouthAmerica = create_SouthAmerica_csp()
    result = SouthAmerica.backtracking_search()
    if result:
        for area, color in sorted(result.items()):
            print("{}: {}".format(area, color))
        print('------------------------------------------')
        SouthAmerica.display_performance()
    else:
        print("No solution found")
    


